In [1]:
# Importing dependencies
import matplotlib.pyplot as plt
import os
import pandas as pd
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine
import json

In [2]:
# Loading homeowrk7.env so that the environment variables can be used in the engine object url
load_dotenv("../project2.env")
# Initializing variables to hold each environmet varaible
username=os.environ.get("USERNAME")
password=os.environ.get("PASSWORD")
host=os.environ.get("HOST")
port=os.environ.get("PORT")
database=os.environ.get("DATABASE")

In [3]:
# Initializing a variable with the created engine
engine = create_engine(f"postgres+psycopg2://{username}:{password}@{host}:{port}/{database}")
# Initializing a variable with the connection resource
connection = engine.connect()

In [4]:
# Initializing a dictionary to hold the fields of the neighborhood_polygon table
fields = {"id": "id INT NOT NULL PRIMARY KEY",
       "field1": "type VARCHAR(17) NULL",
       "field2": "features_type VARCHAR(7) NULL",
       "field3": "features_properties_link VARCHAR(94) NULL",
       "field4": "features_properties_name VARCHAR(28) NULL",
       "field5": "features_geometry_type VARCHAR(12) NULL",
       "field6": "features_geometry_coordinates FLOAT[][] NULL"
      }
# Initializing a dictionary to hold the names of the fields of the neighborhood_polygon table
field_names = {"id": "id",
               "field1": "type",
               "field2": "features_type",
               "field3": "features_properties_link",
               "field4": "features_properties_name",
               "field5": "features_geometry_type",
               "field6": "features_geometry_coordinates"
      }

# Initializing a variable with the string for the neighborhood_polygon table schema
neighborhood_polygons_schema = "{id}, {field1}, {field2}, {field3}, {field4}, {field5}, {field6}".format(**fields)
# Initializing a variable with the string for the neighborhood_polygon table schema
table_data_str = "{id}, {field1}, {field2}, {field3}, {field4}, {field5}, {field6}".format(**field_names)

In [5]:
# Reading in the geoJSON file containing the polygons for each of the San Francisco neighborhoods
file_path = "../geojsons/SF Find Neighborhoods.geojson"
with open(file_path) as jsonfile:
    polygon_json = json.load(jsonfile)

In [6]:
# Using a for-loop to unpack each feature of polygon_json and store it in the neighborhood_polygon table
connection.execute("DROP TABLE IF EXISTS neighborhood_polygons;")
connection.execute("CREATE TABLE IF NOT EXISTS neighborhood_polygons " + f"({neighborhood_polygons_schema})")  
for i in range(len(polygon_json["features"])):
    id = i + 1
    field1 = "'" + polygon_json["type"] + "'"
    field2 = "'" + polygon_json["features"][i]["type"] + "'"
    field3 = "'" + f'{polygon_json["features"][i]["properties"]["link"]}' + "'"
    field4 = "'" + polygon_json["features"][i]["properties"]["name"] + "'"
    field5 = "'" + polygon_json["features"][i]["geometry"]["type"] + "'"
    coordinate_list = polygon_json["features"][i]["geometry"]["coordinates"][0][0]
    field6 = coordinate_list
    #
    connection.execute("INSERT INTO neighborhood_polygons " + f"({table_data_str}) " + f"VALUES ({id}, {field1}, {field2}, {field3}, {field4}, {field5}, ARRAY{coordinate_list})")

In [7]:
# Using a for-loop to insert values into the mortgage_rate_increase
connection.execute("DROP TABLE IF EXISTS mortgage_rate_increase;")
connection.execute("CREATE TABLE IF NOT EXISTS mortgage_rate_increase (id INT NOT NULL PRIMARY KEY, features_properties_name VARCHAR(28) NULL, features_properties_rate_of_increase INT NULL)")
value = 1
for i in range(len(polygon_json["features"])):
    id = i + 1
    field1 = "'" + polygon_json["features"][i]["properties"]["name"] + "'"
    field2 = value
    connection.execute(f"INSERT INTO mortgage_rate_increase (id, features_properties_name, features_properties_rate_of_increase) " + f"VALUES ({id}, {field1}, {field2})")
    value += 1

In [8]:
# Initializing a varaibel to hold a query string that will return the join of all of the columns of the 
# neighborhood_polygons table and the features_properties_rate_of_increase of the mortgage_rate_increase table
query_string = """SELECT neighborhood_polygons.*, mortgage_rate_increase.features_properties_rate_of_increase
                  FROM neighborhood_polygons JOIN mortgage_rate_increase
                  ON neighborhood_polygons.id = mortgage_rate_increase.id
                  ;"""


In [9]:
# Initializing a variable with the returned table
neighborhood_mortgage_rate_df = pd.read_sql(query_string, connection)

In [10]:
# Building a JSON file out of neighborhood_mortgage_rate_df
neighborhood_mortgage_rate_list = []
# Using a for loop to build each geojson feature
for row in neighborhood_mortgage_rate_df.iterrows():
    features_type = row[1][2]
    features_properties_link = row[1][3] 
    features_properties_name = row[1][4] 
    features_properties_rate_of_increase = row[1][7]
    features_geometry_type = row[1][5] 
    features_geometry_coordinates = row[1][6]
    features_geometry_coordinates_list = []
    #
    neighborhood_mortgage_rate_list.append({'type': features_type, 'properties': {'link': features_properties_link, 'name': features_properties_name,
                'MRI': features_properties_rate_of_increase},
 'geometry': {'type': features_geometry_type, 'coordinates': [[features_geometry_coordinates]]}})
#
neighborhood_mortgage_rate_dict = {'type': 'FeatureCollection', 'features': neighborhood_mortgage_rate_list}

In [11]:
# Writing a .geojson file to hold neighborhood_mortgage_rate_dict
with open("../geojsons/neighborhood_polygon_rates.geojson", 'w') as write_file:
    json.dump(neighborhood_mortgage_rate_dict, write_file)